# 🗳️ Titanic Survival Prediction - Ensemble Methods

## 📊 Mục tiêu
- Implement ensemble methods để cải thiện performance
- Sử dụng Voting Classifier và Stacking
- So sánh performance của ensemble vs individual models
- Tạo final model cho submission

## 📋 Nội dung
1. **Data Preparation**
2. **Voting Classifier**
3. **Stacking Classifier**
4. **Bagging Methods**
5. **Ensemble Performance Comparison**
6. **